### 문제 2-4 : 여행 계획 분석기

In [4]:
import pandas as pd
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [5]:
model = ChatOpenAI(temperature=1.0, model_name="gpt-4o")

In [6]:
# data라는 키 값으로 응답의 "테이블의 행을 나타내는 딕셔너리"값들이 들어감
response_schemas = [ 
    ResponseSchema(
        name="response_dict", 
        description="테이블의 행을 나타내는 딕셔너리"
    ),
    ResponseSchema(
        name="keys",
        description="테이블의 행을 나타내는 딕셔너리의 키"
    ),
    ResponseSchema(
        name="shape",
        description="테이블의 크기를 (행, 열)로 나타냄"
    )
]

parser = StructuredOutputParser(
    response_schemas=response_schemas
)

In [7]:
prompt = PromptTemplate(
    template="""
    5개의 필드를 정의하여 정보 추출, 자연어 텍스트에서 핵심 정보 파싱
    
    여행 후기나 계획 텍스트를 입력받아 
    여행지, 기간, 예산, 추천도(1-5점), 
    주요 활동 리스트를 구조화된 형태로 추출하는 시스템을 만드세요.
    
    응답을 JSON 형식으로 반환할건데 JSON 응답 형식의 키를 
    'destination', 
    'duration', 
    'budget', 
    'rating', 
    'activities'
    로 구성해줘
    
    응답 스키마: {format_instructions}
    
    사용자 입력: {query}
    """,
    input_variables=["query"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    }
)
print(prompt.template)
print(prompt.partial_variables)


    5개의 필드를 정의하여 정보 추출, 자연어 텍스트에서 핵심 정보 파싱

    여행 후기나 계획 텍스트를 입력받아 
    여행지, 기간, 예산, 추천도(1-5점), 
    주요 활동 리스트를 구조화된 형태로 추출하는 시스템을 만드세요.

    응답을 JSON 형식으로 반환할건데 JSON 응답 형식의 키를 
    'destination', 
    'duration', 
    'budget', 
    'rating', 
    'activities'
    로 구성해줘

    응답 스키마: {format_instructions}

    사용자 입력: {query}
    
{'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"response_dict": string  // 테이블의 행을 나타내는 딕셔너리\n\t"keys": string  // 테이블의 행을 나타내는 딕셔너리의 키\n\t"shape": string  // 테이블의 크기를 (행, 열)로 나타냄\n}\n```'}


In [8]:
chain = prompt | model | parser

In [9]:
def for_dataframe(user_query):
    try:
        json_res = chain.invoke({"query": user_query})
        # print(f"json_res: {json_res}")
        print("json_res[\"shape\"]: ", json_res["shape"])
        print("json_res[\"keys\"]: ", json_res["keys"])
        
        df = pd.DataFrame(json_res["response_dict"])
        print("\n결과: \n")
        return df
        
    except Exception as e:
        print(f"오류: {e}")
        return None

In [10]:
print("여행 계획 분석기 for pandas.DataFrame")
df_travil_planner = for_dataframe(
    "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. \
        총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. \
            정말 만족스러운 여행이었습니다. \
                5점 만점에 4점 정도 줄 수 있을 것 같아요."
)

df_travil_planner

여행 계획 분석기 for pandas.DataFrame
json_res["shape"]:  (1, 5)
json_res["keys"]:  ['destination', 'duration', 'budget', 'rating', 'activities']

결과: 



destination duration budget  rating  activities
0          부산    2박 3일   30만원       4   해운대 바다 구경
1          부산    2박 3일   30만원       4  자갈치시장 회 먹기
2          부산    2박 3일   30만원       4   감천문화마을 구경

In [78]:
import json

def for_json(user_query):
    try: 
        json_response = chain.invoke({"query": user_query})
        
        result = json_response.get("response_dict", {})
        
        # result["activities"] = json.dumps(result["activities"], ensure_ascii=False, separators=(',', ':'))
        str = '[ '
        if result["activities"]:
            for items in result["activities"]:
                str += items + ', '
        result["activities"] = str[:-2] + " ]"
        result["activities"].replace("\"", "")
        return result

    except Exception as e:
        print(f"오류: {e}")

In [79]:
print("여행 계획 분석기 for JSON")
json_travil_planner = for_json(
    "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. \
        총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. \
            정말 만족스러운 여행이었습니다. \
                5점 만점에 4점 정도 줄 수 있을 것 같아요."
)


print(json.dumps(json_travil_planner, indent=4, ensure_ascii=False))

print(type(json_travil_planner["activities"]))

여행 계획 분석기 for JSON
{
    "destination": "부산",
    "duration": "2박 3일",
    "budget": "30만원",
    "rating": 4,
    "activities": "[ 해운대에서 바다구경, 자갈치시장에서 회 먹기, 감천문화마을 구경 ]"
}
<class 'str'>
